# Práctica 1

## Preparación del entorno

In [1]:
import numpy as np
import keras as ker
import matplotlib.pyplot as plt 
import pandas as pd
import sklearn as sk
import PIL
import os
import random

from PIL import Image
from IPython.display import clear_output # Limpiar output
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import LabelEncoder
from numpy import asarray
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Flatten
from keras.constraints import maxnorm

# SETUP

## Procesamiento de datos

In [2]:
"""
    Primer paso es convertir las imagenes en un array y etiquetarlas.
    Primero de todo, en el siguiente bucle recorreremos todos los ficheros de entrenamiento, convirtiendo cada imagen en una matriz (normalizada / 255)
    Una vez la tenemos normalizada, la añadimos al conjunto de entrenamiento
    Como cada imagen esta ya en una carpeta según su clase, guardamos de que carpeta pertenece la imagen y así la etiquetamos en las salidas esperadas
"""

entradas_entrenamiento = []
salidas_entrenamiento = []
with os.scandir('../Dataset/imgs/train') as ficheros: # Iteramos todo el conjunto de entrenamiento
    for fichero in ficheros:
        path = os.path.join(fichero)
        tipo_clase = path.split("/")
        clase_img = tipo_clase[4]
        print("Leyendo imágenes de la carpeta ",clase_img,end="\r")
        with os.scandir(path) as imagenes:
            for imagen in imagenes:
                img_ruta = os.path.join(imagen)
                img = Image.open(img_ruta)
                img_pixeles = asarray(img) 
                img_normalizada = img_pixeles / 255 # Normalizamos
                entradas_entrenamiento.append(img_normalizada)
                salidas_entrenamiento.append(clase_img)
print("Todas las imágenes del entrenamiento leidas y guardadas")

# Ahora hacemos shuffle y dividimos en entreno y validacion

# Shuffle de ambos a la vez para que las posiciones concuerden con las salidas
shuffle = list(zip(entradas_entrenamiento, salidas_entrenamiento))
random.shuffle(shuffle)
entradas_entrenamiento, salidas_entrenamiento = zip(*shuffle)

# División
entrenamiento_entradas, validacion_entradas = train_test_split(entradas_entrenamiento, test_size = 0.20, shuffle=False)
entrenamiento_salidas, validacion_salidas = train_test_split(salidas_entrenamiento, test_size = 0.20, shuffle=False)

KeyboardInterrupt: 

## Variables a definir

In [ ]:
# No tocar
num_clases = 10
xpixel = 128
ypixel = 96

# Tocar
n_neuronas_conv1 = 32
n_neuronas_conv2 = 32
n_neuronas_conv3 = 32
l_rate = 0.1
epoch = 12 
batch = 128

# Implementación de la red

### Primero realizamos la aumentación y creación de nuevos imputs a partir de las imagenes que tenemos (Augmentation)
Lo realizamos con ImageDataGenerator de keras

In [ ]:
def Augmentation(e_entradas,e_salidas,v_entradas,v_salidas,x,y):
    train_generator = ImageDataGenerator( # Esto lo vamos a tener que poner todo seguido sin comentar para que no salga plagio !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        rescale=1/255.,              # normalize pixel values between 0-1                       > Es posible que esto no sea necesario ya que ya esta normalizado
        brightness_range=[0.1, 0.7], # specify the range in which to decrease/increase brightness
        width_shift_range=0.5,       # shift the width of the image 50%
        rotation_range=0,
        shear_range = 0.2,
        zoom_range=0.2,
        horizontal_flip=True,        # 180 degree flip horizontally
        vertical_flip=True,          # 180 degree flip vertically
        validation_split=0.2        # 15% of the data will be used for validation at end of each epoch
    )
    parametros = {'target_size': (x,y), # CAMBIAR ESTO SEGÚN EL TAMAÑO DE LAS IMAGENES
            'batch_size': 64,
            'n_classes': 10,
            'n_channels': 1,
            'subset':'training',
            'shuffle': True}
    input_entrenamiento = train_generator.flow_from_directory(e_entradas,e_salidas,**parametros)
    input_validacion = train_generator.flow_from_directory(v_entradas,v_salidas,**parametros)
    return input_entrenamiento,input_validacion
entrenamiento, validacion = Augmentation(entrenamiento_entradas,entrenamiento_salidas,validacion_entradas,validacion_salidas,xpixel,ypixel)

### Y ahora creamos la arquitectura de la red convolucional 

In [ ]:
def Modelar_red(n_conv1,n_conv2,n_conv3,x,y,lr):
    model=Sequential()
    # Capa input
    model.add(Conv2D(filters= n_conv1, kernel_size= (3,3),input_shape=(x,y,3),padding='same',activation='relu',kernel_constraint=maxnorm(3))) 
    # Capas convolucionales
    #   >este bloque se puede seguir añadiendo, quiza con menos neuronas, o menos capas convolucionales, pongo dos por dar un ejemplo nada mas
    model.add(Conv2D(n_conv2,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2))) # pooling
    model.add(Dropout(0.2)) # dropout

    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3))) 
    model.add(Conv2D(n_conv3,(3,3),activation='relu',padding='same',kernel_constraint=maxnorm(3)))
    model.add(MaxPooling2D(pool_size=(2,2))) 
    model.add(Dropout(0.2)) 

    # Capa fully-connected
    model.add(Flatten())
    model.add(Dense(512,activation='relu',kernel_constraint=maxnorm(3))) # red fully-connected
    model.add(Dropout(0.5))
    model.add(Dense(num_clases, activation='softmax')) # capa de salida(softmaxx) 

    print(model.summary()) # con esto inspeccionamos el modelo, muy comodo

    # Compilamos
    adam = ker.optimizers.Adam(learning_rate= lr)
    model.compile(loss=ker.loss.categorical_crossentropy,optimizer=adam,metrics=['accuracy'])

    return model
model = Modelar_red(n_neuronas_conv1,n_neuronas_conv2,n_neuronas_conv3,xpixel,ypixel,l_rate)

### Y ahora realizamos el fit para entrenar la red

In [ ]:
def Entrenar(m,e,v,epo,b): #(model,entrenamiento,validacion, epoch,batch)
        m.fit(generator = e,validation_data=v,
                use_multiprocessing=True,workers=6, # Esta parte es para que se separen 6 threads paralelos gracias a fit_generator
                epochs= epo,batch_size = b,verbose=False) 
        return m
model = Entrenar(model,entrenamiento,validacion,epoch,batch)

### Evaluamos el modelo con los datos de test (si hay...)

In [ ]:
def Evaluar(m,e_test,s_test): # model,entrada_test,salida_test
    return m.evaluate(e_test,s_test,verbose=False)
resultado = Evaluar(model, entrada_test, salida_test) # no existen estas variables todavia

### Predecir:

In [ ]:
prediccion = model.predict(x_test) # no existe esta variable todavia
print(prediccion)

# Bibliografía
- https://www.learndatasci.com/tutorials/convolutional-neural-networks-image-classification/
- https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
- https://www.tutorialspoint.com/keras/keras_convolution_neural_network.htm
- https://data-flair.training/blogs/keras-convolution-neural-network/